## Introduction
Motor vehicle crashes remain a significant public safety concern in many cities across the globe. Understanding the underlying causes of traffic accidents is essential for improving road safety, optimizing traffic regulations, and informing policy decisions. With the increasing availability of open data, machine learning offers powerful tools for discovering patterns and predicting outcomes that can lead to actionable insights.

This project uses publicly available crash, vehicle, and people datasets from the City of Chicago to develop a predictive model. The goal is to analyze a wide range of factors — including vehicle information, road conditions, and driver characteristics — to identify the primary contributory cause of each accident. By modeling this problem, we aim to support traffic safety stakeholders in better understanding crash dynamics and designing targeted interventions.

## Business Understanding
Motor vehicle accidents remain a persistent and costly public safety issue in urban centers. Every year, thousands of traffic accidents result in injuries, fatalities, and significant economic losses. Understanding the primary contributory causes of these crashes can help policymakers, urban planners, and safety boards implement targeted interventions to reduce accidents.

#### Objectives
Build a model that can predict the primary contributory cause of a car accident, given information about the car, the people in the car, the road conditions etc.

#### Key Questions
1. What are the most significant factors contributing to vehicle crashes in Chicago?
2. How do weather and lighting conditions impact accident causation patterns?
3. How do driver demographics (age, gender) correlate with accident causes?



## Data Understanding
This project uses three datasets published by City of Chicago (https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if)

* Traffic Crashes-Crashes Data: Shows information about each traffic crash on city streets within the City of Chicago
* Traffic Crashes-Driver/Passenger Data: Contains information about people involved in a crash and if any injuries were sustained
* Traffic Crashes- Vehicle Data: Contains information about vehicles involved in a traffic crash

##### Import the packages

In [1]:
#Import the necessary packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

%matplotlib inline


##### Load the Datasets

In [2]:
# Load crashes dataset
crashes_df=pd.read_csv('.\DATA\Crashes.csv', low_memory=False)
crashes_df.head()

,CRASH_RECORD_ID,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,01184614b08579d6befa7734427e750d0f10e79dc0aa42...,NaN,04/04/2025 01:57:00 AM,15,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,PARKING LOT,...,0.0,0.0,2.0,0.0,1,6,4,41.766021,-87.572442,POINT (-87.572442196614 41.766020695126)
1,01db2b5a81860ebd2fa3f4564ba0de00508847313ce23a...,NaN,04/24/2025 08:40:00 PM,10,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,REAR TO SIDE,PARKING LOT,...,0.0,0.0,2.0,0.0,20,5,4,41.875048,-87.744921,POINT (-87.744921303711 41.875047815678)
2,0583b2385710470c0a1bc80417d095512e3d04d6f26158...,NaN,04/24/2025 05:45:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAWN,TURNING,NOT DIVIDED,...,0.0,0.0,2.0,0.0,5,5,4,41.981238,-87.806997,POINT (-87.806997368434 41.981238161467)
3,05d78a76ea643730a5771ac904b43c56236f332b42db0e...,NaN,04/24/2025 11:13:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,FIXED OBJECT,ALLEY,...,0.0,1.0,0.0,0.0,11,5,4,41.742295,-87.652157,POINT (-87.652156998855 41.742294907932)
4,00a530520c84927490b597a6220ff3f2a3347472ef3086...,NaN,01/15/2025 01:50:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,FIXED OBJECT,RAMP,...,0.0,0.0,1.0,0.0,13,4,1,41.816073,-87.656743,POINT (-87.656742699936 41.816073475493)


In [3]:
# Load Vehicle dataset
Vehicle_df=pd.read_csv('.\DATA\Vehicles.csv', low_memory=False)
Vehicle_df.head()

,CRASH_UNIT_ID,CRASH_RECORD_ID,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,MODEL,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,10,2e31858c0e411f0bdcb337fb7c415aa93763cf2f23e02f...,08/04/2015 12:40:00 PM,1,DRIVER,NaN,10.0,NaN,FORD,Focus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100,e73b35bd7651b0c6693162bee0666db159b28901437009...,07/31/2015 05:50:00 PM,1,DRIVER,NaN,96.0,NaN,NISSAN,Pathfinder,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000,f2b1adeb85a15112e4fb7db74bff440d6ca53ff7a21e10...,09/02/2015 11:45:00 AM,1,DRIVER,NaN,954.0,NaN,FORD,F150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000,15a3e24fce3ce7cd2b02d44013d1a93ff2fbdca80632ec...,10/31/2015 09:30:00 PM,2,DRIVER,NaN,9561.0,NaN,HYUNDAI,SONATA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100000,1d3c178880366c77deaf06b8c3198429112a1c8e8807ed...,11/16/2016 01:00:00 PM,2,PARKED,NaN,96745.0,NaN,"TOYOTA MOTOR COMPANY, LTD.",RAV4 (sport utility),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Load People dataset
People_df=pd.read_csv('.\DATA\People.csv', low_memory=False)
People_df.head()

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,SEX,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,M,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,M,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,NaN,X,...,NaN,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,NaN,X,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,NaN,X,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [ ]:
# Merge the datasets
people_vehicle_df=pd.merge(People_df,Vehicle_df, how='inner', on='CRASH_RECORD_ID')

data_df=pd.merge(crashes_df, people_vehicle_df,  how='inner', on='CRASH_RECORD_ID')

data_df.head()

## Exploratory Data Analysis

In [ ]:
# Take a sample of the data
sample_df = data_df.sample(frac=0.2, random_state=42)
sample_df

In [ ]:
# Make a copy of the data
sample_df=sample_df.copy()

In [ ]:
# Select columns to use
cols_to_keep = [
        'CRASH_DATE', 'POSTED_SPEED_LIMIT', 'WEATHER_CONDITION',
        'LIGHTING_CONDITION', 'ROADWAY_SURFACE_COND', 'CRASH_TYPE',
        'DAMAGE', 'PRIM_CONTRIBUTORY_CAUSE', 'SEX', 'AGE', 'INJURIES_TOTAL',
        'VEHICLE_TYPE', 'MAKE', 'MODEL', 'VEHICLE_YEAR'
    ]
    
data_df = sample_df[cols_to_keep]

In [ ]:
# Check shape of the data
data_df.shape

In [ ]:
# Dataset information
data_df.info()

In [ ]:
#Descriptive statistics
data_df.describe()

##### Missing Values

In [ ]:
# Check for missing values
data_df.isna().sum().sort_values(ascending=False)

In [ ]:
# Fill the missing values with median(Numeric columns)
data_df['AGE'].fillna(data_df['AGE'].median(), inplace=True)
data_df['VEHICLE_YEAR'].fillna(data_df['VEHICLE_YEAR'].median(), inplace=True)

# Categorical columns
cat_cols = data_df.select_dtypes(include='object').columns.tolist()

# Fill in missing values with the most frequent value
for col in cat_cols:
    if data_df[col].isnull().any():  
        most_frequent = data_df[col].mode(dropna=True)[0] 
        data_df[col].fillna(most_frequent, inplace=True)

##### Duplicates

In [ ]:
# check for duplicates
data_df.duplicated().sum()

In [ ]:
# Remove duplicates
data_df.drop_duplicates(inplace=True)

In [ ]:
# Check for the number of unique values in each column
for col in cat_cols:
    print(f"{col}: {data_df[col].nunique()} unique values")

In [ ]:
# Convert categoricals with too many categories to 'OTHER'
for col in ['MAKE', 'MODEL', 'VEHICLE_TYPE']:
        top_values = data_df[col].value_counts().nlargest(10).index
        data_df[col] = data_df[col].where(data_df[col].isin(top_values), 'OTHER')
    
data_df

##### What are the most significant factors contributing to vehicle crashes in Chicago?

In [ ]:
# Select the 5 most common causes
top_causes = data_df['PRIM_CONTRIBUTORY_CAUSE'].value_counts().nlargest(5).index
top_causes

The most frequent contributory cause was 'Unable to determine,' but this category was excluded from the analysis due to its lack of actionable insight.

In [ ]:
# Remove rows with vague/non-actionable contributory causes to improve data quality
data_df = data_df[~data_df['PRIM_CONTRIBUTORY_CAUSE'].isin([
    'UNABLE TO DETERMINE',  # Excluded due to ambiguity
    'NOT APPLICABLE'        # Excluded as it provides no causal insight
])]

In [ ]:
# Visualize the 5 top causes
top_causes = data_df['PRIM_CONTRIBUTORY_CAUSE'].value_counts().nlargest(5)

# Plot
plt.figure(figsize=(10, 6))
# Bar plot
sns.barplot(x=top_causes.values, y=top_causes.index, palette='viridis')

# Set title and labels
plt.title('Top 5 Primary Contributory Causes of Crashes')
plt.xlabel('Number of Crashes')
plt.ylabel('Cause')
plt.show()

##### How do weather conditions impact the likelihood of different types of accidents?


In [ ]:
# Top 5 weather conditions 
top_weather = data_df['WEATHER_CONDITION'].value_counts().nlargest(5).index

# Top 5 contributory causes 
top_causes = data_df['PRIM_CONTRIBUTORY_CAUSE'].value_counts().nlargest(5).reset_index()
top_cause_names = top_causes['index'].tolist() 

# Filter the data
plot_data = data_df[
    (data_df['WEATHER_CONDITION'].isin(top_weather)) & 
    (data_df['PRIM_CONTRIBUTORY_CAUSE'].isin(top_cause_names))
]

# Plot
plt.figure(figsize=(14, 8))
sns.countplot(
    x='WEATHER_CONDITION',
    hue='PRIM_CONTRIBUTORY_CAUSE',
    data=plot_data,
    order=top_weather,
    hue_order=top_cause_names
)

plt.title('Top Contributory Causes by Weather Condition', fontsize=16, pad=20)
plt.xlabel('Weather Condition', fontsize=12)
plt.ylabel('Number of Accidents', fontsize=12)
plt.xticks(rotation=45)
plt.legend(title='Primary Cause', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

##### How does lighting condition affect accident causation patterns?

In [ ]:
data_df['LIGHTING_CONDITION'].unique()

In [ ]:
data_df['LIGHTING_CONDITION'] = data_df['LIGHTING_CONDITION'].replace({
    'DARKNESS, LIGHTED ROAD': 'DARKNESS',
   })

In [ ]:
light_df = data_df[['LIGHTING_CONDITION', 'PRIM_CONTRIBUTORY_CAUSE']].dropna()

In [ ]:
top_lighting = light_df['LIGHTING_CONDITION'].value_counts().nlargest(5).index
top_causes = light_df['PRIM_CONTRIBUTORY_CAUSE'].value_counts().nlargest(5).index

filtered_df = light_df[
    light_df['LIGHTING_CONDITION'].isin(top_lighting) &
    light_df['PRIM_CONTRIBUTORY_CAUSE'].isin(top_causes)
]


In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=filtered_df,
              x='LIGHTING_CONDITION',
              hue='PRIM_CONTRIBUTORY_CAUSE',
              palette='Set3')

plt.title('Top Accident Causes by Lighting Condition')
plt.xlabel('Lighting Condition')
plt.ylabel('Number of Accidents')
plt.xticks(rotation=45)
plt.legend(title='Primary Cause', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

##### How do driver demographics (age, gender) correlate with accident causes?

##### Gender

In [ ]:
# Filter data to include only what is in the top causes
filtered_df = data_df[
    data_df['PRIM_CONTRIBUTORY_CAUSE'].isin(top_causes)
]


In [ ]:
# Plot figure
plt.figure(figsize=(12, 6))

# Countplot
sns.countplot(data=filtered_df, x='SEX', hue='PRIM_CONTRIBUTORY_CAUSE', palette='Set2')

# Set title, labels 
plt.title('Crash Causes by Driver Gender')
plt.xlabel('Driver Gender')
plt.ylabel('Number of Crashes')

# Legend
plt.legend(title='Contributory Cause', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


**Observation**

Gender plays a significant role in road accidents, with males contributing to a much higher number of accidents for all listed causes.
This may suggest that male drivers are either more frequently involved in risky driving behaviors or are more represented in the driving population (or both).

#### Age Distribution

In [ ]:
# Filter data to remove age below 16 and above 100
filtered_df = data_df[
    (data_df['AGE'] >= 16) & 
    (data_df['AGE'] <= 100) & 
    (data_df['PRIM_CONTRIBUTORY_CAUSE'].isin(top_causes))
]


In [ ]:
# Plot figure
plt.figure(figsize=(12, 6))

# Boxplot
sns.boxplot(data=filtered_df, x='PRIM_CONTRIBUTORY_CAUSE', y='AGE', palette='pastel')

# Set title, labels 
plt.title('Driver Age Distribution by Crash Cause')
plt.xlabel('Contributory Cause')
plt.ylabel('Driver Age')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


**Observation**

The data distributions for each category are similar in terms of center and spread.

All groups contain significant outliers, which might warrant further investigation.

No category stands out as significantly different in terms of overall distribution, but the green category shows more variability.

## Feature Engineering

In [ ]:
# Selecting target and features from the dataframe

# Features
X = data_df.drop(['PRIM_CONTRIBUTORY_CAUSE', 'CRASH_DATE'], axis=1)  
# Target
y = data_df['PRIM_CONTRIBUTORY_CAUSE']

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42)

In [ ]:
# Numeric columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
#Categorical columns
categorical_cols = X.select_dtypes(include='object').columns.tolist()


##### Preprocessing
A preprocessing pipeline was created, where numeric columns underwent median imputation and standardization (StandardScaler), while categorical columns were processed using imputation and one-hot encoding

In [ ]:
 # Create transformers
numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])
    
categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    
# Combine transformers
preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_cols),
            ('cat', categorical_transformer, categorical_cols)])
    

## Model Selection and Training

##### Logistic Regression
Logistic regression model was used since we're predicting categories (causes of accidents)

In [ ]:
# Logistic Regression Pipeline
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        max_iter=1000, 
        class_weight='balanced',
        solver='saga',
        random_state=42))
])

# Keep only rows where the target is in top causes
mask = y_train.isin(top_causes)

X_train_top = X_train[mask]
y_train_top = y_train[mask]

# Apply same filter to test data
mask_test = y_test.isin(top_causes)
X_test_top = X_test[mask_test]
y_test_top = y_test[mask_test]


# Model training
lr_pipeline.fit(X_train_top, y_train_top)
y_pred_top = lr_pipeline.predict(X_test_top)


## Model Evaluation and Optimization

In [ ]:

# Accuracy
accuracy = accuracy_score(y_test_top, y_pred_top)
print(f"Accuracy: {accuracy:.3f}")

# Classification report
print("Classification Report:")
print(classification_report(y_test_top, y_pred_top))


**Observation**

The model achieved an overall accuracy of 27.9%. This indicates the model is struggling to accurately classify the primary contributory causes of crashes.
Precision and recall are generally low, suggesting the model has difficulty distinguishing between accident causes.

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test_top, y_pred_top, labels=np.unique(y_test_top))

# Plot
plt.figure(figsize=(10, 6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=y_test_top.unique(), yticklabels=y_test_top.unique(), cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - Logistic Regression")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


**Observation**

The confusion matrix suggests that while the model has reasonable accuracy, there is room for improvement, especially in reducing confusion between certain classes. 


##### Random Forest Classifier

While logistic regression served as a reasonable baseline, its performance on this multi-class classification problem proved limited. To enhance predictive capability, we implemented a Random Forest classifier, which typically handles complex decision boundaries and multi-class scenarios more effectively.

In [ ]:
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ))
])


In [ ]:
# Fit the model
rf_pipeline.fit(X_train_top, y_train_top)

# Predict
y_pred_rf = rf_pipeline.predict(X_test_top)


In [ ]:
# Accuracy
print("Accuracy:", accuracy_score(y_test_top, y_pred_rf))

# Report
print(classification_report(y_test_top, y_pred_rf))

**Observation**

The Random Forest model achieved an accuracy of 34.1%, which is an improvement over the logistic regression baseline (27.9%).
* "Failing to Yield Right-of-Way" had the highest precision (42%) and recall (50%), showing the model was most confident and accurate with this class.
* Classes like "Improper Lane Usage" and "Improper Overtaking/Passing" had low precision and recall, indicating the model struggles to distinguish these.

The model still shows difficulty predicting minority or more ambiguous causes of crashes.

In [ ]:

# Confusion Matrix
cm = confusion_matrix(y_test_top, y_pred_rf, labels=np.unique(y_test_top))

# Plot figure
plt.figure(figsize=(10, 6))

# Heatmap
sns.heatmap(cm, annot=True, fmt='d', xticklabels=y_test_top.unique(), yticklabels=y_test_top.unique(), cmap='YlGnBu')

# Set title, labels
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - Random Forest")

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


**Observation**
* "improper overtaking/passing" has the highest correct predictions (3,376), likely due to its larger representation in the data (class imbalance)

## Model Tuning

Hyperparameter tuning was done using GridSearchCV.

In [ ]:
param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10],  # Regularization strength
    'classifier__solver': ['saga', 'lbfgs'],  # Solvers for multiclass
    'classifier__penalty': ['l2'],  # L1 requires sparse data & saga only
}

grid_search = GridSearchCV(
    lr_pipeline,
    param_grid,
    scoring='f1_weighted',  # Or 'accuracy', 'recall_macro' etc.
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_top, y_train_top)


In [ ]:
print("Best Parameters:", grid_search.best_params_)

# Predict with best estimator
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_top)  # Use X_test_top to match your filtered causes

# Evaluate
from sklearn.metrics import classification_report, accuracy_score

print("Tuned Model Accuracy:", accuracy_score(y_test_top, y_pred))
print("\nClassification Report:\n", classification_report(y_test_top, y_pred))


**Observation**

The best logistic regression parameters were:C=10, penalty='l2', and solver='lbfgs'.
The overall accuracy after tuning was ~27.8%, indicating only a slight improvement from the untuned mode.

The logistic regression model, even after tuning, has limited predictive power

#### Tuning the Random Forest hyperparameters

In [ ]:
# Define the Param grid
param_grid_rf = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [10, 20, None],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2],
}

# Create GridSearchCV
grid_search_rf = GridSearchCV(
    rf_pipeline,
    param_grid=param_grid_rf,
    scoring='f1_weighted',
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit the GridSearchCV
grid_search_rf.fit(X_train_top, y_train_top)

In [ ]:
# Evaluate
print("Best Parameters:", grid_search_rf.best_params_)

# Predict using best model
best_rf_model = grid_search_rf.best_estimator_
y_pred_rf_tuned = best_rf_model.predict(X_test_top)

# Evaluate performance
from sklearn.metrics import classification_report, accuracy_score

print("Tuned Random Forest Accuracy:", accuracy_score(y_test_top, y_pred_rf_tuned))
print("\nClassification Report:\n", classification_report(y_test_top, y_pred_rf_tuned))


**Observation**

While the accuracy dropped slightly from 34.1% to 33.3%, the weighted F1-score remained strong, suggesting the model now treats minor classes more equitably.

#### Feature Importance

In [ ]:

# feature importances from the model
importances = rf_pipeline.named_steps['classifier'].feature_importances_

# Feature names
ohe = rf_pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot']
categorical_feature_names = ohe.get_feature_names_out(categorical_cols)
all_feature_names = list(numeric_cols) + list(categorical_feature_names)

# Create a DataFrame
feat_imp_df = pd.DataFrame({'feature': all_feature_names, 'importance': importances})
feat_imp_df = feat_imp_df.sort_values(by='importance', ascending=False).head(20)

# Plot
plt.figure(figsize=(10,6))
plt.barh(feat_imp_df['feature'], feat_imp_df['importance'])
plt.xlabel('Importance')
plt.title('Top 20 Feature Importances (Random Forest)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


**Observation**

* 'AGE' and 'VEHICLE_YEAR' are by far the most important features.
* Gender (SEX_M/F) has minimal influence, suggesting it’s not a major predictor.
* Vehicle makes show slight variations in importance, possibly reflecting differences in crash rates by brand.
* Weather/lighting conditions  have negligible effects, implying they may not significantly impact predictions

## Conclusion

This project focused on predicting the **Primary Contributory Cause** of vehicle crashes using traffic accident data from Chicago. The workflow involved data preprocessing, exploratory analysis, model building, and hyperparameter tuning using Logistic Regression and Random Forest classifiers.

- The **Top 5 accident causes** were:
  - *Failing to Yield Right-of-Way*
  - *Following Too Closely*
  - *Improper Lane Usage*
  - *Failing to Reduce Speed to Avoid Crash*
  - *Improper Overtaking/Passing*

- **Model Performance**

| Model                | Accuracy | Key Notes |
|---------------------|----------|-----------|
| Logistic Regression (Tuned) | 27.8%   | Better recall for certain causes but low precision overall |
| Random Forest (Tuned)       | 33.3%   | Better balance across classes, but slight accuracy drop after tuning |

- Despite tuning, accuracy remained modest due to:
  - Class imbalance
  - Overlapping feature characteristics
  - Limited discriminative power of available features



## Recommendations

**Stakeholders**
* Driver-Focused Programs: Develop public awareness and training campaigns addressing top causes.
* Targeted interventions; Focus enforcement and road safety improvements on road signage.

**Data Team**
- Handle class imbalance using other techniques
- Try advanced models like **XGBoost** or **LightGBM**
- Evaluate using additional metrics (e.g., ROC AUC, confusion matrices)

While prediction accuracy is moderate, this project highlights how **machine learning** can uncover valuable **patterns in accident causation**, providing data-driven insights for **policy makers, road safety advocates**, and **urban planners** to mitigate crash risks and improve public safety.